In [2]:
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.common.exceptions import TimeoutException

import time
import pandas as pd
import numpy as np


In [3]:
def configure_driver():
    '''Configure the webdriver'''

    #Configurations
    webdriver_path = "./chromedriver.exe" #your webdriver path (chromedriver.exe)

    chrome_options = Options()

    # Turn off Chrome notification and set language to English
    chrome_options.add_argument("--disable-notifications")
    chrome_options.add_experimental_option('prefs', {'intl.accept_languages': 'en,en_US'})

    # Start the service
    service = Service(webdriver_path)

    # Start the driver
    driver = webdriver.Chrome(service=service, options=chrome_options)
    
    return driver


In [4]:
driver = configure_driver()

In [22]:
# Crawling hrefs
# Testing breaking loop
# n = 535
n = 1
check = n

with open('job_hrefs.txt', 'w', encoding='utf-8') as file:
    while(True):
        domain = f'https://careerviet.vn/viec-lam/tat-ca-viec-lam-trang-{n}-vi.html'
        driver.get(domain)
        time.sleep(3)

        try:
            if driver.current_url == 'https://careerviet.vn/viec-lam/ho-chi-minh-l8-vi.html':
                driver.get(domain)
                time.sleep(3)
            else:
                main = driver.find_element(By.CSS_SELECTOR, "div[class='main-slide']") 
        except:
            print("End of page")
            break

        ele = driver.find_elements(By.XPATH, "//div[contains(@class, 'title ')]//h2//a[@class='job_link']")

        for element in ele:
            href = element.get_attribute('href')
            file.write(href + '\n')

        n += 1


End of page


In [5]:
'''
job_title: title of the job
company: name of the company
location: city where the job is
created_date: job's created or updated date
job_categories: list of categories of job
job_type: type of recruiting employee
salary: salary of the job
experience: required experience
level: level of the job
due_date: deadline for applications
job_description: description for the job
requirements: qualifications for the job
others: other information about the job
'''

def crawl(driver):
    try:
        job_title = driver.find_element(By.CSS_SELECTOR, "h1[class='title']").text
    except:
        job_title = ''

    # Xử lý trường hợp không có tên công ty
    try:
        company = driver.find_element(By.CSS_SELECTOR, "a[class='employer job-company-name']").text
    except:
        company = ''

    try:
        location = driver.find_element(By.CSS_SELECTOR, "div[class='map'] p a").text
    except:
        location = ''

    try:
        created_date = driver.find_element(By.XPATH, "//li[strong[contains(text(), 'Ngày cập nhật')]]/p").text
    except:
        created_date = ''
    
    try:
        job_type = driver.find_element(By.XPATH, "//li[strong[contains(text(), 'Hình thức')]]/p").text
    except:
        job_type = ''

    try:
        salary = driver.find_element(By.XPATH, "//li[strong[contains(text(), 'Lương')]]/p").text
    except:
        salary = ''
    
    # Xử lý trường hợp không có phần tử kinh nghiệm
    try:
        experience = driver.find_element(By.XPATH, "//li[strong[contains(text(), 'Kinh nghiệm')]]/p").text
    except:
        experience = ''

    try:
        level = driver.find_element(By.XPATH, "//li[strong[contains(text(), 'Cấp bậc')]]/p").text
    except:
        level = ''

    try:
        due_date = driver.find_element(By.XPATH, "//li[strong[contains(text(), 'Hết hạn nộp')]]/p").text
    except:
        due_date = ''

    job_categories = []
    try:
        categories = driver.find_elements(By.CSS_SELECTOR, "div[class='detail-box has-background'] ul li p a")
        for category in categories:
            job_categories.append(category.text)
    except:
        job_categories = []
    
    try:
        job_benefits = []
        benefits = driver.find_elements(By.XPATH, "//ul[@class='welfare-list']/li")
        [job_benefits.append(b.text) for b in benefits]
    except:
        job_benefits = []
    
    try:
        job_description = driver.find_element(By.XPATH, "//div[h2[contains(text(), 'Mô tả Công việc')]]").text 
    except:
        job_description = ''
    
    try:
        job_requirements = driver.find_element(By.XPATH, "//div[h2[contains(text(), 'Yêu Cầu Công Việc')]]").text
    except:
        job_requirements = ''

    try:
        job_address = driver.find_element(By.XPATH, "//div[h3[contains(text(), 'Địa điểm làm việc')]]").text
    except:
        job_address = ''

    try:
        other_info = driver.find_element(By.XPATH, "//div[h3[contains(text(), 'Thông tin khác')]]").text   
    except:
        other_info = ''

    try:
        job_skills = []
        skills = driver.find_elements(By.XPATH, "//div[@class='job-tags ']/ul/li/a")
        [job_skills.append(skill.text) for skill in skills]
    except:
        job_skills = []

    try:
        # Wait maximum 10s until the <a> tag is visible and clickable
        company_button = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.XPATH, "//ul[@class='tabs-toggle']//li[@id='tabs-job-company']/a"))
        )
        company_button.click()  # Click the <a> tag

        size = WebDriverWait(driver, 10).until(
        EC.visibility_of_element_located(
            (By.XPATH, "//div[@class='content']/ul/li[contains(., ' Qui mô công ty:')]")
            )
        ).text

        company_size = size.split(':')[1].strip()
    except:
        company_size = ''


    result = [
            job_title,
            company, 
            company_size,
            location, 
            created_date, 
            job_categories, 
            job_type, salary, 
            experience, level, 
            due_date, 
            job_benefits, 
            job_description,
            job_requirements,
            job_address,
            other_info,
            skills,
        ]

    return result

In [6]:
# Test crawler
driver.get('https://careerviet.vn/vi/tim-viec-lam/chuyen-vien-chuyen-vien-cao-cap-phong-thuc-day-ban-quan-ly-thuc-thi-trung-tam-trien-khai-san-pham-va-thuc-day-ban-khoi-khdn.35C2012F.html')
crawl(driver)

['Chuyên viên/Chuyên viên cao cấp - Phòng Thúc đẩy bán & Quản lý thực thi - Trung tâm Triển khai sản phẩm và thúc đẩy bán - Khối KHDN',
 'Ngân hàng TMCP Lộc Phát Việt Nam (LPBank)',
 '10.000-19.999',
 'Hồ Chí Minh',
 '10/10/2024',
 ['Bảo hiểm', 'Chứng khoán', 'Dịch vụ khách hàng'],
 'Nhân viên chính thức',
 'Cạnh tranh',
 'Trên 5 Năm',
 'Nhân viên',
 '08/11/2024',
 ['Chế độ bảo hiểm',
  'Du Lịch',
  'Phụ cấp',
  'Chế độ thưởng',
  'Chăm sóc sức khỏe',
  'Đào tạo',
  'Tăng lương',
  'Công tác phí',
  'Nghỉ phép năm'],
 'MÔ TẢ CÔNG VIỆC\n- Hướng dẫn triển khai công tác bán hàng cho đội ngũ bán hàng tại khu vực phụ tráchTheo dõi, giám sát, đôn đốc và thúc đẩy công tác bán hàng của đội ngũ bán hàng tại các ĐVKD tại khu vực phụ trách để đảm bảo hoàn thành kế hoạch kinh doanh theo từng thời kỳ\n- Phối hợp Trung tâm Sản phẩm và Giải pháp xây dựng và triển khai các Chương trình, chính sách thúc đẩy bán theo định hướng từng thời kỳ\n- Đánh giá tình hình triển khai bán tại từng ĐVKD và hiệu suất

In [11]:
import pandas as pd

with open('job_hrefs.txt', 'r', encoding='utf-8') as file:
    job_links = file.readlines()

cols = [
        'job_title', 
        'company', 
        'company_size'
        'location', 
        'created_date', 
        'job_categories', 
        'job_type', 
        'salary', 
        'experience', 
        'level', 
        'due_date', 
        'job_benefits',
        'job_description',
        'job_requirements',
        'job_address',
        'other_info',
        'skills'
    ]

df = pd.DataFrame(columns=cols)

data_list = []
for link in job_links:
    try:
        driver.get(link)
        # Time out exceeded if the page's loading is over 10s
        # Also try to find the title, throw time out and not crawl if the title doesn't exist
        WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, "h1[class='title']")))
        # print(f"{link}{crawl(driver)}\n")
        data_list.append(crawl(driver))
    except TimeoutException:
        print(f'Time out: {link}')
    

Time out: https://careerviet.vn/vi/tim-viec-lam/middle-senior-data-engineer.35C1513B.html



UnboundLocalError: local variable 'skills' referenced before assignment

: 

In [8]:
cols = [
        'job_title', 
        'company', 
        'company_size',
        'location', 
        'created_date', 
        'job_categories', 
        'job_type', 
        'salary', 
        'experience', 
        'level', 
        'due_date', 
        'job_benefits',
        'job_description',
        'job_requirements',
        'job_address',
        'other_info',
        'skills',
    ]
df = pd.DataFrame(data_list, columns=cols)

In [10]:
df

,job_title,company,company_size,location,created_date,job_categories,job_type,salary,experience,level,due_date,job_benefits,job_description,job_requirements,job_address,other_info,skills
0,[G6] _ [BSV] _Tổng Đài Viettel Telecom Quận Tâ...,BellSystem24-Việt Nam,,Hồ Chí Minh,07/10/2024,"[Bưu chính viễn thông, Bán lẻ / Bán sỉ, Kế toá...",Nhân viên chính thức,8 Tr - 15 Tr VND,,Nhân viên,05/11/2024,"[Chế độ thưởng, Đào tạo, Phụ cấp thâm niên, Ng...",MÔ TẢ CÔNG VIỆC\nMô tả công việc:\nTiếp nhận D...,YÊU CẦU CÔNG VIỆC\nYêu cầu công việc:\nTrình đ...,,THÔNG TIN KHÁC\nBằng cấp: Trung học\nĐộ tuổi: ...,[]
1,TUYỂN KINH DOANH SHOWROOM (NỘI THẤT),CÔNG TY TNHH NIPPON INTERIA,,Hồ Chí Minh,07/10/2024,"[Bán hàng / Kinh doanh, Nội ngoại thất, Bất độ...",Nhân viên chính thức,Cạnh tranh,Trên 1 Năm,Nhân viên,05/11/2024,"[Laptop, Chế độ bảo hiểm, Du Lịch, Phụ cấp, Du...",MÔ TẢ CÔNG VIỆC\nMục đích công việc\n- Bán sản...,YÊU CẦU CÔNG VIỆC\n- Nam/Nữ tuổi từ 24 - 35\n-...,ĐỊA ĐIỂM LÀM VIỆC\nHồ Chí Minh\nThành phố Hồ C...,THÔNG TIN KHÁC\nBằng cấp: Trung cấp\nĐộ tuổi: ...,[]
2,Chỉ huy trưởng công trình_Dự án cao tốc,Công ty CP Đầu tư phát triển Đô thị và Khu côn...,,Quảng Ngãi,07/10/2024,"[Kiến trúc, Xây dựng]",Nhân viên chính thức,30 Tr - 40 Tr VND,Trên 5 Năm,Quản lý,15/10/2024,"[Chế độ bảo hiểm, Phụ cấp, Đồng phục, Chế độ t...",MÔ TẢ CÔNG VIỆC\nTriển khai lập kế hoạch thi c...,YÊU CẦU CÔNG VIỆC\nTốt nghiệp Đại học chính qu...,ĐỊA ĐIỂM LÀM VIỆC\nQuảng Ngãi\nPhường Phổ Ninh...,THÔNG TIN KHÁC\nBằng cấp: Cao đẳng\nGiới tính:...,[]
3,Property Manager(Residential),Savills Vietnam,,Hồ Chí Minh,07/10/2024,"[Bảo trì / Sửa chữa, Xây dựng, Bất động sản]",Nhân viên chính thức,Cạnh tranh,5 - 10 Năm,Quản lý,05/11/2024,"[Laptop, Chế độ bảo hiểm, Du Lịch, Phụ cấp, Ch...",MÔ TẢ CÔNG VIỆC\nProperty Manager will be over...,YÊU CẦU CÔNG VIỆC\nAt least 5 years experience...,"ĐỊA ĐIỂM LÀM VIỆC\nHồ Chí Minh\nSơn Kỳ, Tân Ph...",THÔNG TIN KHÁC\nBằng cấp: Cao đẳng\nĐộ tuổi: K...,[]
4,Phú Quốc - Nhân Viên Kỹ Thuật - Mảng ME - Khu ...,BIM Group,,Kiên Giang,07/10/2024,"[Điện / Điện tử / Điện lạnh, Bảo trì / Sửa chữa]",Nhân viên chính thức,8 Tr - 10 Tr VND,Trên 1 Năm,Nhân viên,16/10/2024,"[Laptop, Chế độ bảo hiểm, Du Lịch, Phụ cấp, Ch...",MÔ TẢ CÔNG VIỆC\nQuản lý vận hành các hệ thống...,YÊU CẦU CÔNG VIỆC\nTốt nghiệp Nghề/Trung cấp/C...,"ĐỊA ĐIỂM LÀM VIỆC\nKiên Giang\nDương Tơ, Thành...",THÔNG TIN KHÁC\nBằng cấp: Trung cấp\nĐộ tuổi: ...,[]
5,Marketing Leader (Hai Bà Trưng),CÔNG TY TNHH AZ MỸ-VIỆT,25-99,Hà Nội,07/10/2024,"[Quản lý điều hành, Tiếp thị / Marketing]",Nhân viên chính thức,13 Tr - 15 Tr VND,Trên 5 Năm,Trưởng nhóm / Giám sát,05/11/2024,"[Chế độ bảo hiểm, Du Lịch, Chế độ thưởng, Chăm...",MÔ TẢ CÔNG VIỆC\n1. Hoạch định chiến lược Mark...,YÊU CẦU CÔNG VIỆC\nTốt nghiệp các ngành quản t...,ĐỊA ĐIỂM LÀM VIỆC\nHà Nội\nsố 15 Trần Khát Châ...,THÔNG TIN KHÁC\nBằng cấp: Cao đẳng\nĐộ tuổi: K...,[]
6,Nhân Viên Sales Admin,Công ty cổ phần thương mại Hương Việt,100-499,Hà Nội,07/10/2024,"[Bán lẻ / Bán sỉ, Bán hàng / Kinh doanh, Tiếp ...",Nhân viên chính thức,8 Tr - 10 Tr VND,1 - 2 Năm,Nhân viên,05/11/2024,"[Laptop, Chế độ bảo hiểm, Du Lịch, Du lịch nướ...",MÔ TẢ CÔNG VIỆC\nMô tả công việc:\nTiếp nhận t...,"YÊU CẦU CÔNG VIỆC\nChỉ tuyển nữ, độ tuổi 22-30...",ĐỊA ĐIỂM LÀM VIỆC\nHà Nội\n130 Nguyễn Đức Cảnh...,THÔNG TIN KHÁC\nBằng cấp: Trung học\nGiới tính...,[]
7,Nhân viên kinh doanh (Sóc Trăng),CÔNG TY TNHH THƯƠNG MẠI KỸ THUẬT TRƯỜNG THỊNH,25-99,Sóc Trăng,07/10/2024,"[Tiếp thị / Marketing, Bán hàng / Kinh doanh, ...",Nhân viên chính thức,Cạnh tranh,Lên đến 1 Năm,Nhân viên,05/11/2024,"[Chế độ bảo hiểm, Du Lịch, Đồng phục, Chế độ t...",MÔ TẢ CÔNG VIỆC\nThực hiện các công việc kinh ...,YÊU CẦU CÔNG VIỆC\nCó ít nhất 1 năm kinh nghiệ...,"ĐỊA ĐIỂM LÀM VIỆC\nSóc Trăng\nSố 24, Khóm 3, P...",THÔNG TIN KHÁC\nBằng cấp: Cao đẳng\nĐộ tuổi: K...,[]


In [32]:
df.to_csv('job_data_crv1.csv', index=False)